<a href="https://colab.research.google.com/github/ramziso/twitter_filter/blob/master/Copy_of_Copy_of_Spark_on_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java, Spark, and Findspark
This installs Apache Spark 2.2.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.3.4/spark-2.3.4-bin-hadoop2.7.tgz
!ls -l
!tar xf spark-2.3.4-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://repo1.maven.org/maven2/com/redislabs/spark-redis/2.3.0/spark-redis-2.3.0-jar-with-dependencies.jar

total 221796
drwxr-xr-x 1 root root      4096 Oct 22 15:18 sample_data
-rw-r--r-- 1 root root 227110658 Aug 26 09:00 spark-2.3.4-bin-hadoop2.7.tgz


# Set Environment Variables
Set the locations where Spark and Java are installed.

In [0]:
import os


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.4-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /content/spark-redis-2.3.0-jar-with-dependencies.jar pyspark-shell"

#--packages com.redislabs:spark-redis:2.3.0
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--jars /content/spark-redis-2.3.0.jar pyspark-shell"

In [0]:
!cp /content/spark-redis-2.3.0-jar-with-dependencies.jar /content/spark-2.3.4-bin-hadoop2.7/python/lib/

In [0]:
import sys
sys.path.insert(0, "/content/spark-2.3.4-bin-hadoop2.7/jars")
sys.path.insert(0, "/content/spark-2.3.4-bin-hadoop2.7/python/pyspark")
sys.path.insert(0, os.path.join('SPARK_HOME','/content/spark-redis-2.3.0-jar-with-dependencies.jar'))

# Start a SparkSession
This will start a local Spark session.

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.master("local[*]").config("spark.redis.host", "SG-ramzi-27118.servers.mongodirector.com").config("spark.redis.port", "6379").config("spark.redis.auth", "QLbG0tiG65nKq1JgaPmK3y48tlSzUGYc").config('spark.jars.packages', 'com.redislabs:spark-redis:2.3.0').getOrCreate()

sc = spark.sparkContext

tweet_schema = StructType([
    StructField('lang', StringType(), True),
    StructField('text', StringType(), True)
    
])

In [0]:
!cp $SPARK_HOME/conf/spark-defaults.conf.template $SPARK_HOME/conf/spark-defaults.conf

In [0]:
!echo "spark.jars.packages               com.redislabs:spark-redis:2.3.0" >> $SPARK_HOME/conf/spark-defaults.conf

In [0]:
!cat $SPARK_HOME/conf/spark-defaults.conf

spark.jars.packages               com.redislabs:spark-redis:2.3.0


In [0]:
!cp spark-redis-2.3.0-jar-with-dependencies.jar /content/spark-2.3.4-bin-hadoop2.7/jars
!ls -l /content/spark-2.3.4-bin-hadoop2.7/jars

In [11]:
loadedDf = spark.read.format("org.apache.spark.sql.redis").option("table", "twitter").load()

Py4JJavaError: ignored

In [0]:
loadedDf.printSchema()

# Use Spark!
That's all there is to it - you're ready to use Spark!

In [0]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

/content/spark-2.3.4-bin-hadoop2.7/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [6]:
!pip install redis

     |████████████████████████████████| 71kB 2.7MB/s 


In [0]:
import redis

REDIS_HOST = 'SG-ramzi-27118.servers.mongodirector.com'
REDIS_PORT = 6379
REDIS_LIST = 'ramzi'

# Redis Configuration
redis_password = "QLbG0tiG65nKq1JgaPmK3y48tlSzUGYc"

In [0]:
r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=redis_password, db=0)

In [0]:
import json

class TweetStore:

    # Redis Configuration
    redis_host = "SG-ramzi-27118.servers.mongodirector.com"
    redis_port = 6379
    redis_password = "QLbG0tiG65nKq1JgaPmK3y48tlSzUGYc"
    redis_db = 0
    
    # Tweet Configuration
    redis_key = 'tweets'
    num_tweets = 20

    def __init__(self):
        self.db = r = redis.Redis(
            host=self.redis_host,
            port=self.redis_port,
            password=self.redis_password,
            db=self.redis_db
        )
        self.trim_count = 0

    def push(self, data):
        self.db.lpush(self.redis_key, json.dumps(data))
        self.trim_count += 1

        # Periodically trim the list so it doesn't grow too large.
        if self.trim_count > 100:
            self.db.ltrim(self.redis_key, 0, self.num_tweets)
            self.trim_count = 0

    def tweets(self, limit=15):
        tweets = []

        for item in self.db.lrange(self.redis_key, 0, limit-1):
            tweet_obj = json.loads(item)
            tweets.append(tweet_obj)

        return tweets

In [0]:
r.set('foo','bar')

True

In [91]:
import json
aa = r.lrange('twitter', 0, -1)
type (aa)
# Decode UTF-8 bytes to Unicode, and convert single quotes 
# to double quotes to make it valid JSON
#my_json = aa[:10].decode('utf8').replace("'", '"')
my_json = aa.decode('utf8').replace("'", '"')
print(my_json)
#print('- ' * 20)

# Load the JSON to a Python list & dump it back out as formatted JSON
data = json.loads(my_json)
s = json.dumps(data, indent=4, sort_keys=True)
print(s)

#with open('/content/dump.json', 'w') as f:
#        json.dump(aa, f)


AttributeError: ignored

In [0]:
!pip install redis-dump-load

In [0]:
import json

data = {
    'foo': 'bar',
    'ans': 42
}
r.execute_command('JSON.SET', 'object', '.', json.dumps(data))
reply = json.loads(r.execute_command('JSON.GET', 'object'))

In [10]:
r.get('foo')

b'bar'